In [ ]:
import numpy as np
import simpy
from typing import Optional, List
from dataclasses import dataclass, field, fields
import pandas as pd


In [ ]:
env = simpy.Environment()

# Dataclass as numpy void

In [ ]:
@dataclass
class ProductionOrder_1(np.ndarray):
    _id_counter = 0

    def __new__(
        cls,
        product: str,
        quantity: np.int16 = 1,
        due_date: np.float32 = 0,
        scheduled: np.float32 = 0,
        released: np.float32 = 0,
        priority: np.int8 = 9,
        process_total: np.int16 = 0,
        process_actual: np.int16 = 0,
        finished: np.bool = False,
        finish_at: np.float32 = 0,
        local: str = "backlog",
        id: np.int32 = None,
    ):
        if id is None:
            id = cls._id_counter
            cls._id_counter += 1

        po_dtype = np.dtype(
            [
                ("id", np.int32),  # Order ID
                ("product", "U50"),  # Product name
                ("quantity", np.int16),  # Product quantity
                ("due_date", np.float32),  # Due date
                ("scheduled", np.float32),  # Scheduled to release
                ("released", np.float32),  # Released time to shopfloor
                ("priority", np.int8),  # Order priority
                ("process_total", np.int16),  # Total process that order needs
                ("process_finished", np.int16),  # Actual process that order is
                ("finished", np.bool),  # If order is finished
                ("finish_at", np.float32),  # Finished at
                ("status", "U50"),  # Local on the production line
            ]
        )

        raw_data = (
            id,
            product,
            quantity,
            due_date,
            scheduled,
            released,
            priority,
            process_total,
            process_actual,
            finished,
            finish_at,
            local,
        )

        arr = np.array([raw_data], dtype=po_dtype)[0]

        return arr.view(cls)

    @classmethod
    def reset_counter(cls, value=0):
        cls._id_counter = value

store_1 = simpy.FilterStore(env)

for i in range(0,100):  
    productionOrder = ProductionOrder_1("produto1",quantity=1)
    productionOrder["priority"] = int(np.random.random()*10)
    store_1.put(productionOrder)

def _get_order_resource_queue_1(method):
        match method:
            case "fifo":
                productionOrderId = store_1.items[0]["id"]

            case "toc_penetration":
                productionOrderId = np.sort(store_1.items, order="priority")[0]["id"]
        
        return productionOrderId

def teste_01():

    # Get order from queue
    productionOrderId = _get_order_resource_queue_1("toc_penetration")
    productionOrder: ProductionOrder_1 = yield store_1.get(
        lambda item: item["id"] == productionOrderId
    )
    
    productionOrder["status"] = "teste"
    product = productionOrder["product"]
    process = productionOrder["process_finished"]
    productionOrder["process_finished"] += 1
    productionOrder["status"] = "queue"
    
    yield store_1.put(productionOrder)

In [ ]:
%%timeit
env.process(teste_01())
env.run()

# Np.void arrays

In [ ]:
def _create_production_orders(size=100000):
        po_dtype = [
            ("id", np.int32),  # Order ID
            ("used", np.bool),  # If slod is used
            ("product", "U12"),  # Product name
            ("quantity", np.int16),  # Product quantity
            ("due_date", np.float64),  # Due date
            ("scheduled", np.float64),  # Scheduled to release
            ("released", np.float64),  # Released time to shopfloor
            ("priority", np.int8),  # Order priority
            ("process_total", np.int16),  # Total process that order needs
            ("process_actual", np.int16),  # Actual process that order is
            ("finish_at", np.float64),  # Finished at
            ("status", str),  # If order is finished
            ("resource", str),  # Orders actual resource
        ]
        po_orders = np.zeros(size, dtype=po_dtype)
        po_orders["id"] = range(0, len(po_orders))
        return po_orders

orders = _create_production_orders()

orders["priority"] = np.round(np.random.random(size=100000)*10,0)

store_2 = simpy.FilterStore(env)
store_3 = simpy.FilterStore(env)

for i in range(0,100):  
    store_2.put(i)
    store_3.put(i)

def _get_order_resource_queue_2(method):
        match method:
            case "fifo":
                productionOrdersIds = store_2.items
                productionOrderId = orders[
                    orders["id"] == productionOrdersIds[0]
                ]["id"]

            case "toc_penetration":
                productionOrderId = np.sort(orders, order="priority")[0]
        
        return productionOrderId

def _get_order_resource_queue_3(method):
        match method:
            case "fifo":
                productionOrdersIds = store_3.items
                productionOrderId = orders[
                    orders["id"] == productionOrdersIds[0]
                ]["id"]

            case "toc_penetration":
                productionOrderId = np.sort(orders, order="priority")[0]
                
        
        return productionOrderId

def teste_02():

    # Get order from queue
    productionOrderId = _get_order_resource_queue_2("fifo")
    productionOrderId = yield store_2.get(
        lambda item: item == productionOrderId
    )
    
    po_mask = orders["id"] == productionOrderId
    orders[po_mask]["status"] = "teste"
    product = orders[po_mask]["product"]
    process = orders[po_mask]["process_actual"]
    orders[po_mask]["process_actual"] += 1
    orders[po_mask]["status"] = "queue"

    yield store_2.put(productionOrderId)
    

def teste_03():

    # Get order from queue
    productionOrderId = _get_order_resource_queue_3("fifo")
    productionOrderId = yield store_3.get(
        lambda item: item == productionOrderId
    )
    
    po_mask = orders["id"] == productionOrderId
    productionOrder = orders[po_mask].copy()
    
    productionOrder["status"] = "teste"
    product = productionOrder["product"]
    process = productionOrder["process_actual"]
    productionOrder["process_actual"] += 1
    productionOrder["status"] = "queue"
    
    yield store_3.put(productionOrderId)
    orders[po_mask] = productionOrder

    

In [ ]:
%%timeit
env.process(teste_02())
env.run()

In [ ]:
%%timeit
env.process(teste_03())
env.run()

# Structured Dataclass

In [ ]:
@dataclass
class ProductionOrder_4:
    product: str
    quantity: int
    schedule: Optional[float] = None
    released: Optional[int] = None
    duedate: Optional[float] = None
    finished: Optional[bool] = None
    priority: Optional[int] = None
    process_total: Optional[int] = None
    process_finished: Optional[int] = None
    status: Optional[str] = None
    id: int = field(init=False)

    _next_id = 1

    def __post_init__(self):
        self.id = ProductionOrder_4._next_id
        ProductionOrder_4._next_id += 1

    def to_dict(self) -> dict:
        keys = [
            "product",
            "quantity",
            "schedule",
            "released",
            "duedate",
            "finished",
            "priority",
            "process_total",
            "process_finished",
            "id",
        ]

        dict_tmp = {key: self.__dict__[key] for key in keys if key in self.__dict__}
        return dict_tmp

store_4 = simpy.FilterStore(env)

for i in range(0,100):  
    productionOrder = ProductionOrder_4("produto1",1, process_finished=0)
    productionOrder.priority = int(np.random.random()*10)
    productionOrder.duedate = np.random.randint(10,20)
    store_4.put(productionOrder)

def _get_order_resource_queue_4(method):
        match method:
            case "fifo":
                productionOrderId = store_4.items[0].id

            case "toc_penetration":
                now = 15
                priorities = [
                     [po.id, po.duedate - now] for po in store_4.items
                ]
                productionOrderId = sorted(priorities, key= lambda x: x[1])[0][0]

        return productionOrderId


def teste_04():

    # Get order from queue
    productionOrderId = _get_order_resource_queue_4("toc_penetration")
    productionOrder: ProductionOrder_4 = yield store_4.get(
        lambda item: item.id == productionOrderId
    )
    
    productionOrder.status = "teste"
    product = productionOrder.product
    process = productionOrder.process_finished
    productionOrder.process_finished += 1
    productionOrder.status = "queue"
    
    yield store_4.put(productionOrder)

In [ ]:
# %%timeit
env.process(teste_04())
env.run()

# FullDataframe

In [ ]:
from pathlib import Path
import yaml

with open(Path("../config/products.yaml"), "r") as f:
    products_config = yaml.safe_load(f)

with open(Path("../config/resources.yaml"), "r") as f:
    resources_config = yaml.safe_load(f)

product_str_ids = { key: i for i,key in enumerate(list(products_config.keys()))}
product_ids_str = { i: key for i,key in enumerate(list(products_config.keys()))}
resources_str_ids = { key: i for i,key in enumerate(list(resources_config.keys()))}
resources_ids_str = { i: key for i,key in enumerate(list(resources_config.keys()))}


In [ ]:
size = 20000
orders_id = np.array(range(0,size))
orders_product = np.zeros(shape=(size))
orders_quantity = np.zeros(shape=(size))
orders_schedule = np.zeros(shape=(size))
orders_released = np.zeros(shape=(size))
orders_duedate = np.zeros(shape=(size))
orders_finished = np.zeros(shape=(size))
orders_priority = np.zeros(shape=(size))
orders_process_total = np.zeros(shape=(size))
orders_process_actual = np.zeros(shape=(size))
orders_status = np.zeros(shape=(size))

ORDER_ID = 0

def create_order(
    order_id,
    product,
    quantity,
    schedule = 0,
    released = 0,
    duedate = 0,
    finished = 0,
    priority = 0,
    process_total = 0,
    process_actual = 0,
    status = 0,
    
):
    mask = orders_id == order_id
    
    orders_product[mask] = product_str_ids[product]
    orders_quantity[mask] = quantity
    orders_schedule[mask] = schedule
    orders_released[mask] = released
    orders_duedate[mask] = duedate
    orders_finished[mask] = finished
    orders_priority[mask] = priority
    orders_process_total[mask] = process_total
    orders_process_actual[mask] = process_actual
    orders_status[mask] = status

    order_id +=1

    return order_id





In [ ]:
store_5 = simpy.FilterStore(env)

for i in range(0,100):  
    po_id = create_order(ORDER_ID, "produto02", 2)
    ORDER_ID = po_id
    store_5.put(po_id)

def _get_order_resource_queue_5(method):
        match method:
            case "fifo":
                productionOrderId = store_5.items[0]

            case "toc_penetration":
                now = 15
                productionOrderIds = store_5.items
                
                orders_priority[productionOrderIds] = now - orders_duedate[productionOrderIds]
                productionOrderId = np.argsort(orders_priority)[0]
     
        return productionOrderId


def teste_05():

    # Get order from queue
    productionOrderId = _get_order_resource_queue_5("toc_penetration")
    productionOrderId = yield store_5.get(
        lambda item: item == productionOrderId
    )
    mask = orders_id == productionOrderId
    orders_status[mask] = 12
    product = product_ids_str[int(orders_product[mask][0])]
    process = orders_process_actual[mask]
    orders_process_actual[mask] +=1
    orders_status[mask] = 5
        
    yield store_5.put(productionOrderId)

In [ ]:
%%timeit
env.process(teste_05())
env.run()

In [ ]:
orders

In [ ]:
%%timeit
env.process(teste_01())
env.run()

In [ ]:
# %%timeit
env.process(teste_02())
env.run()

In [ ]:
%%timeit
env.process(teste_03())
env.run()

In [ ]:
%%timeit
env.process(teste_04())
env.run()

In [ ]:
# %%timeit
env.process(teste_05())
env.run()

In [ ]:
fields(store_5.items[0])[0].name

In [ ]:
productionOrders = sorted(store_2.items, key= lambda x: x.priority)
productionOrders[0].id

In [ ]:
# orders.sort(0)
orders.sort("priority")
orders

In [ ]:
orders[(orders["id"]>4) & (orders["id"]<11)]["quantity"].sum()

In [ ]:
mask = np.isin(orders["id"], [1,2,3])
mask
# orders[orders["id"]]

In [ ]:
class ProductionOrder(np.void):
    def __new__(cls, size):
        po_dtype = [
            ("id", np.int32),  # Order ID
            ("used", np.bool),  # If slod is used
            ("product", "U12"),  # Product name
            ("quantity", np.int16),  # Product quantity
            ("due_date", np.float64),  # Due date
            ("scheduled", np.float64),  # Scheduled to release
            ("released", np.float64),  # Released time to shopfloor
            ("priority", np.int8),  # Order priority
            ("process_total", np.int16),  # Total process that order needs
            ("process_actual", np.int16),  # Actual process that order is
            ("finished", np.bool),  # If order is finished
            ("finish_at", np.float64),  # Finished at
        ]
        obj = np.zeros(size, dtype=po_dtype)
        return obj

In [ ]:
a = ProductionOrder(1000)

In [ ]:
a = np.array(store_2.items, dtype=ProductionOrder)
a[a.item() == 6]


In [ ]:
po_dtype = [
            ("id", np.int32),  # Order ID
            ("used", np.bool),  # If slod is used
            ("product", "U12"),  # Product name
            ("quantity", np.int16),  # Product quantity
            ("due_date", np.float64),  # Due date
            ("scheduled", np.float64),  # Scheduled to release
            ("released", np.float64),  # Released time to shopfloor
            ("priority", np.int8),  # Order priority
            ("process_total", np.int16),  # Total process that order needs
            ("process_actual", np.int16),  # Actual process that order is
            ("finished", np.bool),  # If order is finished
            ("finish_at", np.float64),  # Finished at
        ]
abc = np.void(0,dtype=po_dtype)

In [ ]:
abc.dtype

In [ ]:
order["product"] = "abc"
order["quantity"] = 5
order["due_date"] = 10.05

id = order["id"]

orders[orders["id"]==id] = order

orders